# PRÁTICA GUIADA: Naive Bayes

## Introdução

Nesta prática vamos implementar um classificador do tipo Naive Bayes usando a biblioteca scikit-learn.

Cada parâmetro de NaiveBayes representa a probabilidade de pertencer a certa classe com determinado valor da feature $X_{k}$. 
No caso da classificação de um texto, o que é medido é a probabilidade de o texto pertencer a determinada classe dependendo da aparição ou não de determinada palavra. O produto de todas essas probabilidades é a probabilidade final de pertencer à classe.  

O que diferencia cada algoritmo de Naive Bayes é a distribuição adotada por eles para o processo de geração de cada uma das classes. Em sklearn temos:
1. GaussianNaiveBayes
2. MultinomialNaiveBayes
3. BernoulliNaiveBayes


## Gaussian Naive Bayes

Geramos classes com uma distribuição gaussiana. 


In [0]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.datasets import make_blobs


X, y = make_blobs(100, 2, centers=2, random_state=2, cluster_std=1.5)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='RdBu');

Vamos aplicar a este conjunto de dados um modelo do tipo Gaussian Naive Bayes.

In [0]:
# Ajustamos o modelo com os dados gerados
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X, y);

In [0]:
# -6 e -14 são os limites inferiores dos dados gerados pela função make_blobs
# 14 e 18 são os intervalos ocupados por cada uma das variáveis geradas
# Queremos gerar um espaço de pontos que abranja toda a área dos dados originais.

rng = np.random.RandomState(0)
X_nueva = [-6, -14] + [14, 18] * rng.rand(2000, 2)
y_predicha = model.predict(X_nueva)

Depois, vemos a classificação definida pelo modelo sobre todo o espaço de dados, descrito pelos 2000 puntos gerados acima. 


In [0]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='RdBu')
lim = plt.axis()
plt.scatter(X_nueva[:, 0], X_nueva[:, 1], c=y_predicha, s=50, cmap='RdBu', alpha=0.1)
plt.axis(lim);

No gráfico anterior observa-se o “decision boundary” do modelo surgido da hipótese de uma distribuição gaussiana sobre cada uma das classes nos dados de treinamento. <br>

### Cálculo de probabilidades

Uma das vantagens deste modelo é que ele nos permite obter a probabilidade de pertença a uma classe ou outra para cada ponto do domínio.

In [0]:
yprob = model.predict_proba(X_nueva)
yprob[-8:].round(2)


## Classificação de texto

Para o seguinte exemplo, vamos baixar o conjunto de dados 20 newsgroup, que contém e-mails com pedidos de informações a diferentes empresas de mídia digital sobre diferentes assuntos.

Para mais informações sobre o conjunto de dados, consultar: http://qwone.com/~jason/20Newsgroups/

### Importamos os dados

In [0]:
from sklearn.datasets import fetch_20newsgroups

mis_datos = fetch_20newsgroups()
type(mis_datos)

In [0]:
# Exploramos o objeto e vemos que o dataframe não tem nomes de colunas
mis_datos.keys()

In [0]:
print(type(mis_datos.data),'El corpus de textos es una lista')
print(type(mis_datos.data[0]),'Cada elemento de la lista es un string')

In [0]:
# Vejamos o primeiro texto do corpus
print(mis_datos.data[0])

In [0]:
# Vejamos agora em quais classes os dados são divididos

mis_datos.target_names

#### Split train-test e simplificação do dataframe

Para facilitar o problema de classificação, vamos ficar só com algumas das categorias que contêm os dados. A classe Bunch tem um método especialmente arquitetado para isso:


In [0]:
categories = ['talk.religion.misc', 'soc.religion.christian',
'sci.space', 'comp.graphics']
train = fetch_20newsgroups(subset='train', categories=categories)
test = fetch_20newsgroups(subset='test', categories=categories)

In [0]:
len(train.data)

In [0]:
len(test.data)

#### Feature Engineering

Para poder implementar um modelo de previsão, precisamos transformar cada um dos documentos de texto pertencentes ao corpus em uma matriz de features.

Para isso, vamos utilizar o algoritmo de scikit-learn `TfidfVectorizer()`. Ele tenta vetorizar um corpus de textos contando a ocorrência de cada palavra em cada documento. Primeiro, procuram-se todas as palavras existentes no corpus, e depois é computada a quantidade de aparições em cada documento. Esse processo é denominado "word count".

O problema dessa abordagem é a existência de palavras que aparecem com muita frequência em um texto, mas contribuem com pouco valor para a classificação: projeta-se que palavras como “a”, “o” ou “do” apareçam muitas vezes em todos os documentos. 

O que o TfidfVectorizer() faz é calcular a frequência de aparição de um termo em um documento ponderada segundo a aparição no restante do corpus a fim dar mais peso aos termos que diferenciam cada documento dos outros.

Mais informações em http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction
e sobre os detalhes de implementação em: http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html


In [0]:
# Geramos o modelo e o aplicamos aos dados de treinamento

from sklearn.feature_extraction.text import TfidfVectorizer
modelo_tfidf = TfidfVectorizer()
X_train = modelo_tfidf.fit_transform(train.data)

In [0]:
# Vejamos a transformação feita sobre os dados
print(len(train.data))
print(X_train.shape)

In [0]:
# Vejamos que tipo de dados X contém
type(X_train)

In [0]:
# A matriz dispersa conserva unicamente os valores diferentes de 0.
X_train.size

In [0]:
# Quando transformamos a matriz em array, a quantidade de elementos é multiplicada várias vezes
X_train.toarray().size

In [0]:
# Vejamos as primeiras e últimas features construídas pelo modelo
modelo_tfidf.get_feature_names()[0:30]

In [0]:
modelo_tfidf.get_feature_names()[-30:]

#### Treinamento de um classificador Naive Bayes

Com a matriz construída para os dados de treinamento, vamos treinar um classificador utilizando o modelo MultinomialNaiveBayes()

In [0]:
from sklearn.naive_bayes import MultinomialNB
modelo_NB = MultinomialNB()
modelo_NB.fit(X_train, train.target)

#### Previsão

Com os parâmetros do modelo já calculados, vamos fazer previsões sobre os dados de teste. 
Como vetorizamos os dados de teste? Temos que usar aquele modelo tf-idf com que nós tínhamos transformado os dados de treinamento!



In [0]:
X_test = modelo_tfidf.transform(test.data)

In [0]:
# Também é gerada uma matriz esparsa
type(X_test)

In [0]:
# Com as features de teste transformadas calculamos as labels previstas
labels_predichas = modelo_NB.predict(X_test)

In [0]:
labels_predichas

In [0]:
# As labels estão ligadas às tags del objeto Bunch original
train.target_names

### Matriz de confusão

A seguir, criamos o gráfico de una matriz de confusão entre a previsão e as verdadeiras labels em teste

In [0]:
from sklearn.metrics import confusion_matrix
% matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns


In [0]:
# Primeiro, calculamos a accuracy geral do modelo
from sklearn.metrics import accuracy_score
accuracy_score(test.target, labels_predichas)

In [0]:
# Imprimimos a matriz de confusão
mat = confusion_matrix(test.target, labels_predichas)
mat.shape

In [0]:
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
xticklabels=train.target_names, yticklabels=train.target_names)plt.xlabel('Etiquetas verdaderas')
plt.xlabel('Etiquetas predichas');

#### Conclusão

Mesmo com um modelo muito simples como o Naive Bayes, nós podemos classificar um corpus de texto com bastante precisão.

Na matriz de confusão podemos observar algo que era esperável: os correios sobre religião em geral se confundem com aqueles específicos sobre a religião cristã, já que empregam um vocabulário semelhante.

Um ponto interessante a considerar é que agora temos um classificador de strings. Podemos colocar qualquer string nele para classificá-la de forma automática. Naturalmente, vamos ter que fazer isso em inglês. Somente isolemos tudo em uma única função:

In [0]:
def predict_category(s,train=train):
    x = modelo_tfidf.transform([s]) # chamamos o vetorizador que usamos (tem que ser o mesmo)
    pred = modelo_NB.predict(x) # fazemos o predict, usando o modelo que fitamos.
    return train.target_names[pred[0]] # retornamos a categoria correspondente à previsão

Vamos provar:

In [0]:
predict_category('sending a payload to the ISS')

In [0]:
predict_category('discussing islam vs atheism')

In [0]:
predict_category('determining the screen resolution')

É preciso lembrar que isso não tem qualquer sofisticação: trata-se simplesmente de um modelo probabilístico (de hipóteses bastante simples) para a frequência ponderada de cada palavra em uma string. Contudo, obteremos um nível de accuracy bastante razoável e resultados bastante efetivos.